In [ ]:
"""
Created on March 2024

Author: Fenia Psomouli
"""

In [2]:
#Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.colors import ListedColormap, BoundaryNorm
from PIL import Image
from tqdm import tqdm
import os
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.enums import Resampling
from skimage.transform import resize
from google.cloud import storage

In [3]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Mount Google Drive to Google Colab
from google.colab import drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive', force_remount=True)


In [ ]:
draining = '/content/drive/MyDrive/Thesis/Data/Draining64'
refreezing = '/content/drive/MyDrive/Thesis/Data/Refreezing64'

In [ ]:
#Create lists for the appropriate paths. Most functions in the following code use a list as input.
if os.path.exists(draining):
    # Initialize an empty list to hold the file paths
    file_pathsD = []

    # Iterate over the list of files and directories
    for filename in os.listdir(draining):
        # Create the full path and add it to the list
        file_pathD = os.path.join(draining, filename)
        file_pathsD.append(file_pathD)

if os.path.exists(refreezing):
    file_pathsR = []

    for filename in os.listdir(refreezing):
        file_pathR = os.path.join(refreezing, filename)
        file_pathsR.append(file_pathR)

In [ ]:
np.save('/content/drive/MyDrive/Thesis/Data/test_dataset_corrected_image_removed.npy', np.array(test_s, dtype=object), allow_pickle=True)


### Data Normalization

In [ ]:
def normalize_bands(input_path, output_path):
    with rasterio.open(input_path) as src:
        meta = src.meta
        all_data = []

        print(f"Processing file: {input_path}")

        # Read and stack first 30 bands
        for b in range(1, src.count + 1):  # src.count should be 31, so range goes to 30
            band_data = src.read(b).astype(np.float32)
            all_data.append(band_data)

        # Convert list to numpy array for processing
        all_data_stack = np.stack(all_data, axis=0)

        # Calculate the 5th and 95th percentiles across all bands
        p5, p95 = np.percentile(all_data_stack, (5, 95))
        print(f"Global percentiles (5th, 95th): ({p5}, {p95})")

        normalized_data = []
        # Normalize each band
        for b, band_data in enumerate(all_data, start=1):
            band_data_clipped = np.clip(band_data, p5, p95)
            band_data_normalized = (band_data_clipped - p5) / (p95 - p5)
            normalized_data.append(band_data_normalized)
            print(f"Band {b}: Normalized Min {band_data_normalized.min()}, Normalized Max {band_data_normalized.max()}")

        # Add the last band unchanged
        last_band = src.read(src.count)
        normalized_data.append(last_band)

        # Update the metadata to include the count of all bands
        meta.update(dtype=rasterio.float32, count=len(normalized_data))

        # Write all bands to new file
        with rasterio.open(output_path, 'w', **meta) as dst:
            for b, data in enumerate(normalized_data, start=1):
                dst.write(data, b)
                print(f"Written band {b} to {output_path}")

        print(f"Finished processing {input_path}")

In [ ]:
def process_multiple_images(file_paths, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for input_path in file_paths:
        directory, file_name = os.path.split(input_path)
        base_name, ext = os.path.splitext(file_name)
        output_file_name = f"{base_name}_normalized{ext}"
        output_path = os.path.join(output_dir, output_file_name)
        normalize_bands(input_path, output_path)
        print(f"Processed and normalized: {output_file_name}")


file_paths = file_pathsD
output_dir = "/content/drive/MyDrive/Thesis/Data/Big_boxes_test_set_normalized"
process_multiple_images(file_paths, output_dir)


### Sensitivity Analysis
#### Creating 10 perturbations from the original testing dataset to perform a sensitivity analysis.
#### Temporal perturbations:


1.   Shuffled Sequence dataset
2.   Repeated Timestep dataset

#### Spatial perturbations:


1.   Decreased Intensity dataset
2.   Increased Intensity dataset
3.   Zoomed In dataset
4.   Zoomed Out dataset
5.   Shifted Left variation 1 dataset
6.   Shifted Left variation 2 dataset
7.   Flipped dataset
8.   Randomly Shuffled Pixels dataset







In [ ]:
# Original Testing Dataset paths
path_list = ['/content/drive/MyDrive/Thesis/Data/Draining64norm/S1_clipped_DrainingEvents_2021_Greenland_67.4022445855227_-49.35524789531343_127_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Draining64norm/S1_clipped_DrainingEvents_2021_Greenland_67.62318859348166_-49.37701494267799_127_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Draining64norm/S1_clipped_DrainingEvents_2021_Greenland_67.46494847923525_-48.696636402041364_127_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Draining64norm/S1_clipped_DrainingEvents_2021_Greenland_66.85861327481095_-48.984696639437644_90_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Draining64norm/S1_clipped_DrainingEvents_2021_Greenland_79.30897225508468_-64.9923231172314_90_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Draining64norm/S1_clipped_DrainingEvents_2021_Greenland_66.91919464143304_-49.41516706981185_127_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Draining64norm/S1_clipped_DrainingEvents_2021_Greenland_67.23196491447491_-49.7056998233567_90_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Draining64norm/S1_clipped_DrainingEvents_2021_Greenland_63.86005506647149_-49.27486146500474_127_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Draining64norm/S1_clipped_DrainingEvents_2021_Greenland_65.34988532296397_-49.66971254641218_127_normalized.tif',

'/content/drive/MyDrive/Thesis/Data/Refreezing64norm/S1_clipped_RefreezingEvents_2021_Greenland_72.61025622675221_-52.871182157667_90_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Refreezing64norm/S1_clipped_RefreezingEvents_2021_Greenland_66.34466173392667_-48.68970250223261_127_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Refreezing64norm/S1_clipped_RefreezingEvents_2021_Greenland_68.64807287470676_-49.291752534637965_90_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Refreezing64norm/S1_clipped_RefreezingEvents_2021_Greenland_65.1913107780621_-48.343689742817695_127_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Refreezing64norm/S1_clipped_RefreezingEvents_2021_Greenland_65.10544189502883_-48.98050501617494_127_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Refreezing64norm/S1_clipped_RefreezingEvents_2021_Greenland_62.36352721842573_-47.784411369726584_90_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Refreezing64norm/S1_clipped_RefreezingEvents_2021_Greenland_65.61827859844682_-48.81403338333206_90_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Refreezing64norm/S1_clipped_RefreezingEvents_2021_Greenland_78.80587414399459_-22.086895190752152_74_normalized.tif',
'/content/drive/MyDrive/Thesis/Data/Refreezing64norm/S1_clipped_RefreezingEvents_2021_Greenland_79.1692596896075_-24.006484530388164_74_normalized.tif',
 '/content/drive/MyDrive/Thesis/Data/Refreezing64norm/S1_clipped_RefreezingEvents_2021_Greenland_62.36352721842573_-47.784411369726584_83_normalized.tif']

### Temporal Perturbations


In [ ]:
#1.Shuffled Sequence dataset
def shuffle_bands(bands):
    """
    Shuffle the first 30 bands randomly, keeping the 31st band unchanged.
    """
    bands_to_shuffle = bands[:30].copy()
    np.random.shuffle(bands_to_shuffle)
    shuffled_bands = np.concatenate((bands_to_shuffle, bands[30:31]), axis=0)
    return shuffled_bands

def process_geotiff(file_path):
    """
    Process a single GeoTIFF file: read, shuffle the first 30 bands, and return both original and modified images.
    """
    with rasterio.open(file_path) as src:
        bands = [src.read(i + 1) for i in range(src.count)]

    original_images = np.array(bands)
    shuffled_images = shuffle_bands(original_images)

    return original_images, shuffled_images

def visualize_images(original_images, shuffled_images, output_dir, file_name, num_bands=31):
    """
    Visualize original and shuffled images side by side.
    """
    fig, axes = plt.subplots(2, num_bands, figsize=(num_bands * 3, 6))

    for i in range(num_bands):
        axes[0, i].imshow(original_images[i], cmap='gray')
        axes[0, i].set_title(f'Original Band {i+1}')
        axes[0, i].axis('off')

        axes[1, i].imshow(shuffled_images[i], cmap='gray')
        axes[1, i].set_title(f'Shuffled Band {i+1}')
        axes[1, i].axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{file_name}_comparison.png'))
    plt.show()

def save_modified_geotiff(original_file_path, shuffled_images, output_path):
    """
    Save the shuffled images as a new GeoTIFF file.
    """
    with rasterio.open(original_file_path) as src:
        profile = src.profile

        with rasterio.open(output_path, 'w', **profile) as dst:
            for i in range(shuffled_images.shape[0]):
                dst.write(shuffled_images[i], i + 1)

def main(path_list, output_dir):
    for file_path in path_list:
        # Process each GeoTIFF file
        original_images, shuffled_images = process_geotiff(file_path)

        # Visualize the images
        file_name = os.path.basename(file_path).replace('.tif', '')
        visualize_images(original_images, shuffled_images, output_dir, file_name)

        # Save the modified images
        output_path = os.path.join(output_dir, f'{file_name}_shuffled.tif')
        save_modified_geotiff(file_path, shuffled_images, output_path)


output_dir = '/content/drive/MyDrive/Thesis/Data/test_within_entry_sequence_shuffle'
main(path_list, output_dir)

In [ ]:
#2.Repeated Timestep dataset
def create_variations(bands):
    """
    Create variations where each new entry consists of one image repeated 30 times and the ground truth as the 31st band.
    """
    ground_truth = bands[-1]
    new_entries = []

    for i in range(len(bands) - 1):
        new_entry = np.tile(bands[i], (30, 1, 1))
        new_entry = np.concatenate((new_entry, [ground_truth]), axis=0)
        new_entries.append(new_entry)

    return new_entries

def process_geotiff(file_path):
    """
    Process a single GeoTIFF file: read and create variations.
    """
    with rasterio.open(file_path) as src:
        bands = [src.read(i) for i in range(1, src.count + 1)]

    original_images = np.array(bands)
    variations = create_variations(original_images)

    return variations

def save_modified_geotiff(original_file_path, variations, output_dir):
    """
    Save the variations as new GeoTIFF files.
    """
    with rasterio.open(original_file_path) as src:
        profile = src.profile
        for i, variation in enumerate(variations):
            variation_path = os.path.join(output_dir, f'{os.path.basename(original_file_path).replace(".tif", "")}_variation_{i+1}.tif')
            with rasterio.open(variation_path, 'w', **profile) as dst:
                for j in range(variation.shape[0]):
                    dst.write(variation[j], j + 1)

def visualize_variations(variations, num_entries=5):
    """
    Visualize the first few entries of the variations.
    """
    for i in range(min(num_entries, len(variations))):
        fig, axes = plt.subplots(2, 31, figsize=(31 * 2, 6))

        for j in range(31):
            axes[0, j].imshow(variations[i][j], cmap='gray')
            axes[0, j].set_title(f'Var {i+1} Band {j+1}')
            axes[0, j].axis('off')

            if j < 30:
                axes[1, j].imshow(variations[i][j], cmap='gray')
            else:
                axes[1, j].imshow(variations[i][j], cmap='gray')
            axes[1, j].set_title(f'Var {i+1} Band {j+1}')
            axes[1, j].axis('off')

        plt.tight_layout()
        plt.show()

def main(path_list, output_dir):
    for file_path in path_list:
        # Process each GeoTIFF file
        variations = process_geotiff(file_path)

        # Visualize the variations
        visualize_variations(variations)

        # Save the modified images
        save_modified_geotiff(file_path, variations, output_dir)


output_dir = '/content/drive/MyDrive/Thesis/Data/test_fixed_image_timestep'
main(path_list, output_dir)


### Spatial perturbations

In [ ]:
#1.Decreased Intensity dataset
def adjust_intensity(image, reduction_value):
    """
    Decrease the intensity of the image by subtracting a constant value.
    """
    adjusted_image = np.clip(image - reduction_value, 0, 1)
    return adjusted_image

def process_geotiff(file_path, reduction_value):
    """
    Process a single GeoTIFF file: read, adjust intensity of first 30 bands, and return both original and modified images.
    """
    with rasterio.open(file_path) as src:
        bands = [src.read(i + 1) for i in range(src.count)]

    original_images = np.array(bands)
    modified_images = np.array([
        adjust_intensity(band, reduction_value) if i < 30 else band
        for i, band in enumerate(bands)
    ])

    return original_images, modified_images

def visualize_images(original_images, modified_images, num_bands=31):
    """
    Visualize original and modified images side by side.
    """
    fig, axes = plt.subplots(num_bands, 2, figsize=(10, num_bands * 3))

    for i in range(num_bands):
        axes[i, 0].imshow(original_images[i], cmap='gray')
        axes[i, 0].set_title(f'Original Band {i+1}')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(modified_images[i], cmap='gray')
        axes[i, 1].set_title(f'Modified Band {i+1}')
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

def save_modified_geotiff(original_file_path, modified_images, output_path):
    """
    Save the modified images as a new GeoTIFF file.
    """
    with rasterio.open(original_file_path) as src:
        profile = src.profile

        with rasterio.open(output_path, 'w', **profile) as dst:
            for i in range(modified_images.shape[0]):
                dst.write(modified_images[i], i + 1)

def main(path_list, reduction_value, output_dir):
    for file_path in path_list:
        # Process each GeoTIFF file
        original_images, modified_images = process_geotiff(file_path, reduction_value)

        # Visualize the images
        visualize_images(original_images, modified_images)

        # Save the modified images
        file_name = os.path.basename(file_path)
        output_path = os.path.join(output_dir, file_name)
        save_modified_geotiff(file_path, modified_images, output_path)

reduction_value = 0.2
output_dir = '/content/drive/MyDrive/Thesis/Data/test_decreased_intensity'
main(path_list, reduction_value, output_dir)


In [ ]:
#2. Increased Intensity dataset
def increase_intensity(image, factor):
    """
    Increase the intensiry of the image by a given factor.
    """
    return np.clip(image * factor, 0, 1)

def process_geotiff(file_path, brightness_factor):
    """
    Process a single GeoTIFF file: read, increase intensity of first 30 bands, and return both original and modified images.
    """
    with rasterio.open(file_path) as src:
        bands = [src.read(i + 1) for i in range(src.count)]

    original_images = np.array(bands)
    modified_images = np.array([
        increase_intensity(band, brightness_factor) if i < 30 else band
        for i, band in enumerate(bands)
    ])

    return original_images, modified_images

def visualize_images(original_images, modified_images, num_bands=31):
    """
    Visualize original and modified images side by side.
    """
    fig, axes = plt.subplots(num_bands, 2, figsize=(10, num_bands * 3))

    for i in range(num_bands):
        axes[i, 0].imshow(original_images[i], cmap='gray')
        axes[i, 0].set_title(f'Original Band {i+1}')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(modified_images[i], cmap='gray')
        axes[i, 1].set_title(f'Modified Band {i+1}')
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

def save_modified_geotiff(original_file_path, modified_images, output_path):
    """
    Save the modified images as a new GeoTIFF file.
    """
    with rasterio.open(original_file_path) as src:
        profile = src.profile

        with rasterio.open(output_path, 'w', **profile) as dst:
            for i in range(modified_images.shape[0]):
                dst.write(modified_images[i], i + 1)

def main(path_list, brightness_factor, output_dir):
    for file_path in path_list:
        # Process each GeoTIFF file
        original_images, modified_images = process_geotiff(file_path, brightness_factor)

        # Visualize the images
        visualize_images(original_images, modified_images)

        # Save the modified images
        file_name = os.path.basename(file_path)
        output_path = os.path.join(output_dir, file_name)
        save_modified_geotiff(file_path, modified_images, output_path)


brightness_factor = 1.5
output_dir = '/content/drive/MyDrive/Thesis/Data/test_increased_intensity'
main(path_list, brightness_factor, output_dir)


In [ ]:
#3, 4.Zoomed in and Zoomed out dataset
def zoom_in(image, zoom_factor):
    """
    Zoom into the center of the image by the given zoom factor.
    """
    height, width = image.shape
    new_height = int(height / zoom_factor)
    new_width = int(width / zoom_factor)

    # Calculate the cropping box
    start_row = (height - new_height) // 2
    start_col = (width - new_width) // 2

    # Crop and resize to original dimensions
    cropped_image = image[start_row:start_row + new_height, start_col:start_col + new_width]
    zoomed_image = resize(cropped_image, (height, width), mode='reflect', anti_aliasing=True)

    return zoomed_image

def process_geotiff(file_path, zoom_factor):
    """
    Process a single GeoTIFF file: read, apply zoom, and return both original and zoomed images.
    """
    with rasterio.open(file_path) as src:
        bands = [src.read(i + 1) for i in range(src.count)]

    original_images = np.array(bands)
    zoomed_images = np.array([zoom_in(band, zoom_factor) for band in bands])

    return original_images, zoomed_images

def visualize_images(original_images, zoomed_images, num_bands=31):
    """
    Visualize original and zoomed images side by side.
    """
    fig, axes = plt.subplots(num_bands, 2, figsize=(10, num_bands * 3))

    for i in range(num_bands):
        axes[i, 0].imshow(original_images[i], cmap='gray')
        axes[i, 0].set_title(f'Original Band {i+1}')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(zoomed_images[i], cmap='gray')
        axes[i, 1].set_title(f'Zoomed Band {i+1}')
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

def save_zoomed_geotiff(original_file_path, zoomed_images, output_path):
    """
    Save the zoomed images as a new GeoTIFF file.
    """
    with rasterio.open(original_file_path) as src:
        profile = src.profile
        profile.update(height=zoomed_images.shape[1], width=zoomed_images.shape[2])

        with rasterio.open(output_path, 'w', **profile) as dst:
            for i in range(zoomed_images.shape[0]):
                dst.write(zoomed_images[i], i + 1)

def main(path_list, zoom_factor, output_dir):
    for file_path in path_list:
        # Process each GeoTIFF file
        original_images, zoomed_images = process_geotiff(file_path, zoom_factor)

        # Visualize the images
        visualize_images(original_images, zoomed_images)

        # Save the zoomed images
        file_name = os.path.basename(file_path)
        output_path = os.path.join(output_dir, file_name)
        save_zoomed_geotiff(file_path, zoomed_images, output_path)


zoom_factor = 2
output_dir = '/content/drive/MyDrive/Thesis/Data/test_zoomed_in'
main(path_list, zoom_factor, output_dir)


In [ ]:
#5.Shifted Left variation 1 dataset
def shift_left(image, shift_pixels):
    """
    Shift the image to the left by a given number of pixels.
    """
    height, width = image.shape
    shifted_image = np.zeros_like(image)
    shifted_image[:, :width - shift_pixels] = image[:, shift_pixels:]
    shifted_image[:, width - shift_pixels:] = image[:, :shift_pixels]

    return shifted_image

def process_geotiff(file_path, shift_pixels):
    """
    Process a single GeoTIFF file: read, shift left all 31 bands, and return both original and modified images.
    """
    with rasterio.open(file_path) as src:
        bands = [src.read(i + 1) for i in range(src.count)]

    original_images = np.array(bands)
    modified_images = np.array([shift_left(band, shift_pixels) for band in bands])

    return original_images, modified_images

def visualize_images(original_images, modified_images, num_bands=31):
    """
    Visualize original and modified images side by side.
    """
    fig, axes = plt.subplots(num_bands, 2, figsize=(10, num_bands * 3))

    for i in range(num_bands):
        axes[i, 0].imshow(original_images[i], cmap='gray')
        axes[i, 0].set_title(f'Original Band {i+1}')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(modified_images[i], cmap='gray')
        axes[i, 1].set_title(f'Shifted Band {i+1}')
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

def save_modified_geotiff(original_file_path, modified_images, output_path):
    """
    Save the modified images as a new GeoTIFF file.
    """
    with rasterio.open(original_file_path) as src:
        profile = src.profile

        with rasterio.open(output_path, 'w', **profile) as dst:
            for i in range(modified_images.shape[0]):
                dst.write(modified_images[i], i + 1)

def main(path_list, shift_pixels, output_dir):
    for file_path in path_list:
        # Process each GeoTIFF file
        original_images, modified_images = process_geotiff(file_path, shift_pixels)

        # Visualize the images
        visualize_images(original_images, modified_images)

        # Save the modified images
        file_name = os.path.basename(file_path)
        output_path = os.path.join(output_dir, file_name)
        save_modified_geotiff(file_path, modified_images, output_path)



shift_pixels = 10  # number of pixels to shift to the left
output_dir = '/content/drive/MyDrive/Thesis/Data/test_shifted_left'
main(path_list, shift_pixels, output_dir)


In [ ]:
#6.Shifted Left variation 2 dataset
def shift_left(image, shift_pixels):
    """
    Shift the image to the left by a given number of pixels, filling the right with zeros.
    """
    height, width = image.shape
    shifted_image = np.zeros_like(image)
    shifted_image[:, :width - shift_pixels] = image[:, shift_pixels:]

    return shifted_image

def process_geotiff(file_path, shift_pixels):
    """
    Process a single GeoTIFF file: read, shift left all 31 bands, and return both original and modified images.
    """
    with rasterio.open(file_path) as src:
        bands = [src.read(i + 1) for i in range(src.count)]

    original_images = np.array(bands)
    modified_images = np.array([shift_left(band, shift_pixels) for band in bands])

    return original_images, modified_images

def visualize_images(original_images, modified_images, num_bands=31):
    """
    Visualize original and modified images side by side.
    """
    fig, axes = plt.subplots(num_bands, 2, figsize=(10, num_bands * 3))

    for i in range(num_bands):
        axes[i, 0].imshow(original_images[i], cmap='gray')
        axes[i, 0].set_title(f'Original Band {i+1}')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(modified_images[i], cmap='gray')
        axes[i, 1].set_title(f'Shifted Band {i+1}')
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

def save_modified_geotiff(original_file_path, modified_images, output_path):
    """
    Save the modified images as a new GeoTIFF file.
    """
    with rasterio.open(original_file_path) as src:
        profile = src.profile

        with rasterio.open(output_path, 'w', **profile) as dst:
            for i in range(modified_images.shape[0]):
                dst.write(modified_images[i], i + 1)

def main(path_list, shift_pixels, output_dir):
    for file_path in path_list:
        # Process each GeoTIFF file
        original_images, modified_images = process_geotiff(file_path, shift_pixels)

        # Visualize the images
        visualize_images(original_images, modified_images)

        # Save the modified images
        file_name = os.path.basename(file_path)
        output_path = os.path.join(output_dir, file_name)
        save_modified_geotiff(file_path, modified_images, output_path)



shift_pixels = 10  # number of pixels to shift to the left
output_dir = '/content/drive/MyDrive/Thesis/Data/test_shifted_left_black'
main(path_list, shift_pixels, output_dir)

In [ ]:
#7.Flipped dataset
def flip_upside_down(image):
    """
    Flip the image upside down.
    """
    return np.flipud(image)

def process_geotiff(file_path):
    """
    Process a single GeoTIFF file: read, flip, and return both original and flipped images.
    """
    with rasterio.open(file_path) as src:
        bands = [src.read(i + 1) for i in range(src.count)]

    original_images = np.array(bands)
    flipped_images = np.array([flip_upside_down(band) for band in bands])

    return original_images, flipped_images

def visualize_images(original_images, flipped_images, num_bands=31):
    """
    Visualize original and flipped images side by side.
    """
    fig, axes = plt.subplots(num_bands, 2, figsize=(10, num_bands * 3))

    for i in range(num_bands):
        axes[i, 0].imshow(original_images[i], cmap='gray')
        axes[i, 0].set_title(f'Original Band {i+1}')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(flipped_images[i], cmap='gray')
        axes[i, 1].set_title(f'Flipped Band {i+1}')
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

def save_flipped_geotiff(original_file_path, flipped_images, output_path):
    """
    Save the flipped images as a new GeoTIFF file.
    """
    with rasterio.open(original_file_path) as src:
        profile = src.profile

        with rasterio.open(output_path, 'w', **profile) as dst:
            for i in range(flipped_images.shape[0]):
                dst.write(flipped_images[i], i + 1)

def main(path_list, output_dir):
    for file_path in path_list:
        # Process each GeoTIFF file
        original_images, flipped_images = process_geotiff(file_path)

        # Visualize the images
        visualize_images(original_images, flipped_images)

        # Save the flipped images
        file_name = os.path.basename(file_path)
        output_path = os.path.join(output_dir, file_name)
        save_flipped_geotiff(file_path, flipped_images, output_path)


output_dir = '/content/drive/MyDrive/Thesis/Data/test_flipped'
main(path_list, output_dir)

In [ ]:
#8.Randomly Shuffled Pixels dataset
def shuffle_pixels(bands):
    """
    Shuffle the pixels within each band randomly.
    """
    shuffled_bands = []
    for band in bands:
        flat_band = band.flatten()
        np.random.shuffle(flat_band)
        shuffled_band = flat_band.reshape(band.shape)
        shuffled_bands.append(shuffled_band)
    return np.array(shuffled_bands)

def process_geotiff(file_path):
    """
    Process a single GeoTIFF file: read, shuffle the pixels in each band, and return both original and modified images.
    """
    with rasterio.open(file_path) as src:
        bands = [src.read(i + 1) for i in range(src.count)]

    original_images = np.array(bands)
    shuffled_images = shuffle_pixels(original_images)

    return original_images, shuffled_images

def visualize_images(original_images, shuffled_images, output_dir, file_name, num_bands=31):
    """
    Visualize original and shuffled images side by side.
    """
    fig, axes = plt.subplots(2, num_bands, figsize=(num_bands * 3, 6))

    for i in range(num_bands):
        axes[0, i].imshow(original_images[i], cmap='gray')
        axes[0, i].set_title(f'Original Band {i+1}')
        axes[0, i].axis('off')

        axes[1, i].imshow(shuffled_images[i], cmap='gray')
        axes[1, i].set_title(f'Shuffled Band {i+1}')
        axes[1, i].axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{file_name}_comparison.png'))
    plt.show()

def save_modified_geotiff(original_file_path, shuffled_images, output_path):
    """
    Save the shuffled images as a new GeoTIFF file.
    """
    with rasterio.open(original_file_path) as src:
        profile = src.profile

        with rasterio.open(output_path, 'w', **profile) as dst:
            for i in range(shuffled_images.shape[0]):
                dst.write(shuffled_images[i], i + 1)

def main(path_list, output_dir):
    for file_path in path_list:
        # Process each GeoTIFF file
        original_images, shuffled_images = process_geotiff(file_path)

        # Visualize the images
        file_name = os.path.basename(file_path).replace('.tif', '')
        visualize_images(original_images, shuffled_images, output_dir, file_name)

        # Save the modified images
        output_path = os.path.join(output_dir, f'{file_name}_shuffled.tif')
        save_modified_geotiff(file_path, shuffled_images, output_path)


output_dir = '/content/drive/MyDrive/Thesis/Data/test_within_shuffled_pixels'
main(path_list, output_dir)
